## 井点数据预处理


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
file_well = "../../data/well_group.xlsx"
data_well = pd.read_excel(file_well, sheet_name="Sheet1")

data_well.head()

In [ ]:
# 只保留excel的A~C, J, K, AB~AR列
# # 获取所有列名
# all_columns = data_well.columns
# # 选择需要保留的列
# selected_columns = list(all_columns[0:3])  # A~C列
# selected_columns.extend(list(all_columns[9:11]))  # J, K列
# selected_columns.extend(list(all_columns[27:29]))  # AB, AC列

selected_columns = data_well.columns

# 只保留需要的列，并删除层位是P0的行
filtered_data = data_well[selected_columns]
filtered_data = filtered_data[filtered_data["Surface"] != "P0"]
print("筛选后的列：", filtered_data.columns.tolist())
print("筛选后的数据形状：", filtered_data.shape)
print("\n筛选后的数据前5行：")
print(filtered_data.head())

In [ ]:
# 统计"Well"这一列有多少个值，并处理每个Well
well_column = filtered_data.columns[4]  # 井名在第4列
sand_thickness_column = filtered_data.columns[5]  # 砂厚在第5列
sand_ratio_column = filtered_data.columns[6]  # 砂地比在第6列

unique_wells = filtered_data[well_column].unique()
print(f"\n共有 {len(unique_wells)} 个不同的井点值")

# 存储处理后的数据
final_data = pd.DataFrame(columns=filtered_data.columns)

# 要删除的井点列表
wells_to_delete = []

# 循环处理每个井点
for well in unique_wells:
    # 1. 删除指定井点
    if well in wells_to_delete:
        print(f"\n井点 {well} 在指定删除列表中，已删除。")
        continue

    # 获取当前井的数据
    well_data = filtered_data[filtered_data[well_column] == well]

    # # 2. 删除少于5行的井点
    # if len(well_data) < 10:
    #     print(f"\n井点 {well} 只有 {len(well_data)} 行数据，已删除。")
    #     continue

    # 3. 检查砂厚和砂地比列的异常值比例
    sand_thickness_anomaly_ratio = (well_data[sand_thickness_column] == -999).mean()
    sand_ratio_anomaly_ratio = (well_data[sand_ratio_column] == -999).mean()

    # 如果砂厚或砂地比的异常值比例超过70%，则删除该井
    if sand_thickness_anomaly_ratio > 0.7 or sand_ratio_anomaly_ratio > 0.7:
        print(
            f"\n井点 {well} 的砂厚异常值比例为 {sand_thickness_anomaly_ratio:.2%}，砂地比异常值比例为 {sand_ratio_anomaly_ratio:.2%}，已删除。"
        )
        continue

    # 保留符合条件的井点数据
    final_data = pd.concat([final_data, well_data], ignore_index=True)
    print(f"\n井点 {well} 有 {len(well_data)} 行数据，已保留。前5行数据：")
    print(well_data.head())

print(f"\n处理后总共保留了 {len(final_data)} 行数据，包含 {len(final_data[well_column].unique())} 个井点")

# 保存处理后的数据
final_data.to_excel("../../data/well_group_processed.xlsx", index=False)